In [1]:
import pandas as pd

df_results = pd.read_csv("../data/results.csv")
df_drivers = pd.read_csv("../data/drivers.csv")
df_constructors = pd.read_csv("../data/constructors.csv")
df_races = pd.read_csv("../data/races.csv")

In [2]:
df = df_results.merge(df_drivers, on='driverId', how='left', suffixes=('', '_driver'))

In [3]:
df = df.merge(df_constructors, on='constructorId', how='left', suffixes=('', '_constructor'))

In [4]:
df = df.merge(df_races, on='raceId', how='left', suffixes=('', '_race'))

In [5]:
print(df.columns)

Index(['resultId', 'raceId', 'driverId', 'constructorId', 'number', 'grid',
       'position', 'positionText', 'positionOrder', 'points', 'laps', 'time',
       'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'statusId', 'driverRef', 'number_driver', 'code',
       'forename', 'surname', 'dob', 'nationality', 'url', 'constructorRef',
       'name', 'nationality_constructor', 'url_constructor', 'year', 'round',
       'circuitId', 'name_race', 'date', 'time_race', 'url_race', 'fp1_date',
       'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time',
       'quali_date', 'quali_time', 'sprint_date', 'sprint_time'],
      dtype='object')


In [6]:
df['name_driver'] = df['forename'] + ' ' + df['surname']

df_basic = df[[
    'raceId',
    'year',
    'round',
    'driverId',
    'constructorId',
    'grid',
    'positionOrder',
    'points',
    'name_driver',
    'name',
    'circuitId'
]].copy()

In [7]:
df_basic = df_basic[df_basic['positionOrder'].notna()]

In [8]:
#last 5 races performance

df_basic.sort_values(by=['driverId', 'year', 'round'])

df_basic['driver_avg_finish_last_5'] = (
    df_basic.groupby('driverId')['positionOrder']
            .rolling(window=5, min_periods=1)
            .mean()
            .reset_index(level=0, drop=True)
)

df_basic['driver_points_last_5'] = (
    df_basic.groupby('driverId')['points']
            .rolling(window=5, min_periods=1)
            .sum()
            .reset_index(level=0, drop=True)
)

df_basic['driver_avg_finish_last_5'] = df_basic['driver_avg_finish_last_5'].fillna(
    df_basic['driver_avg_finish_last_5'].mean()
)
df_basic['driver_points_last_5'] = df_basic['driver_points_last_5'].fillna(0)

In [9]:
#teammate performance comparison

df_teammate = df_basic[['raceId','constructorId','driverId','grid','positionOrder','points']]

df_basic = df_basic.merge(
    df_teammate,
    on=['raceId','constructorId'],
    suffixes=('', '_teammate')
)

df_basic = df_basic[df_basic['driverId'] != df_basic['driverId_teammate']]

df_basic['grid_diff_teammate'] = df_basic['grid'] - df_basic['grid_teammate']
df_basic['finish_diff_teammate'] = df_basic['positionOrder'] - df_basic['positionOrder_teammate']
df_basic['points_diff_teammate'] = df_basic['points'] - df_basic['points_teammate']

df_basic = df_basic.drop(columns=['driverId_teammate','grid_teammate','positionOrder_teammate','points_teammate'])


In [10]:
#Circuit specific info

driver_circuit_avg = df_basic.groupby(['driverId','circuitId'])['positionOrder'].mean().reset_index()
driver_circuit_avg.rename(columns={'positionOrder':'driver_circuit_avg_finish'}, inplace=True)

df_basic = df_basic.merge(driver_circuit_avg, on=['driverId','circuitId'], how='left')

overall_driver_avg = df_basic.groupby('driverId')['positionOrder'].mean().reset_index()
overall_driver_avg.rename(columns={'positionOrder':'driver_overall_avg_finish'}, inplace=True)

df_basic = df_basic.merge(overall_driver_avg, on='driverId', how='left')
df_basic['driver_circuit_avg_finish'] = df_basic['driver_circuit_avg_finish'].fillna(
    df_basic['driver_overall_avg_finish']
)

In [11]:
from sklearn.preprocessing import LabelEncoder

le_driver = LabelEncoder()
le_constructor = LabelEncoder()
le_circuit = LabelEncoder()

df_basic['driver_encoded'] = le_driver.fit_transform(df_basic['driverId'])
df_basic['constructor_encoded'] = le_constructor.fit_transform(df_basic['constructorId'])
df_basic['circuit_encoded'] = le_circuit.fit_transform(df_basic['circuitId'])

In [12]:
features = df_basic[[
    'year',
    'round',
    'grid',
    'driver_encoded',
    'constructor_encoded',
    'circuit_encoded',
    'driver_avg_finish_last_5',
    'driver_points_last_5',
    'grid_diff_teammate',
    'finish_diff_teammate',
    'points_diff_teammate',
    'driver_circuit_avg_finish'
]]

target = df_basic['positionOrder']

X = features
y = target

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.2, random_state=42
)

In [ ]:
#random forest

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

rf_model = RandomForestRegressor(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"Baseline Random Forest Performance:")
print(f"MAE (mean absolute error): {mae:.2f}")
print(f"RMSE (root mean squared error): {rmse:.2f}")
print(f"R^2 score: {r2:.2f}")

Baseline Random Forest Performance:
MAE (mean absolute error): 1.19
RMSE (root mean squared error): 2.12
R^2 score: 0.93


In [ ]:
#XGBoost regressor

'''
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

xgb_model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.1,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred_xgb)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
r2 = r2_score(y_test, y_pred_xgb)

print(f"XGBoost Performance:")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R^2: {r2:.2f}")
'''

In [ ]:
'''

import matplotlib.pyplot as plt

importances = xgb_model.feature_importances_

feature_names = ['year', 'round', 'grid', 'driver_encoded', 'constructor_encoded', 'circuit_encoded', 'driver_avg_finish_last_5', 'driver_points_last_5','grid_diff_teammate', 'finish_diff_teammate', 'points_diff_teammate', 'driver_circuit_avg_finish' ]

plt.figure(figsize=(8,5))
plt.barh(feature_names, importances, color='skyblue')
plt.xlabel("Importance")
plt.title("XGBoost Feature importance")
plt.gca().invert_yaxis()
plt.show()
'''

NameError: name 'xgb_model' is not defined